In [1]:
# Imports
import pandas as pd
import httpx
import asyncio
from typing import Dict, List, Optional


In [2]:
# API Base URL
OPENF1_BASE_URL = "https://api.openf1.org/v1"


In [3]:
async def fetch_session_key() -> Optional[str]:
    """
    Fetch the session_key for the Bahrain Grand Prix 2023 Race session.
    
    Returns:
        Optional[str]: The session_key if found, None otherwise
    """
    async with httpx.AsyncClient() as client:
        try:
            # Step 1: Find the meeting_key for Bahrain 2023
            print("Step 1: Finding Bahrain 2023 meeting...")
            meetings_response = await client.get(f"{OPENF1_BASE_URL}/meetings?year=2023")
            meetings_response.raise_for_status()
            meetings_data = meetings_response.json()
            
            # Find Bahrain meeting - FIXED: Using 'in' instead of exact match
            bahrain_meeting = None
            for meeting in meetings_data:
                if 'bahrain' in meeting.get('meeting_name', '').lower():
                    bahrain_meeting = meeting
                    break
            
            if not bahrain_meeting:
                print("Error: Bahrain meeting not found in 2023")
                return None
            
            meeting_key = bahrain_meeting['meeting_key']
            print(f"Found Bahrain meeting with key: {meeting_key}")
            
            # Step 2: Find the Race session for this meeting
            print("\nStep 2: Finding Race session...")
            sessions_response = await client.get(f"{OPENF1_BASE_URL}/sessions?meeting_key={meeting_key}")
            sessions_response.raise_for_status()
            sessions_data = sessions_response.json()
            
            # Find Race session
            race_session = None
            for session in sessions_data:
                if session.get('session_name', '').lower() == 'race':
                    race_session = session
                    break
            
            if not race_session:
                print("Error: Race session not found for Bahrain 2023")
                return None
            
            session_key = race_session['session_key']
            print(f"Found Race session with key: {session_key}")
            
            return session_key
            
        except httpx.HTTPStatusError as e:
            print(f"HTTP error occurred: {e}")
            return None
        except Exception as e:
            print(f"Error occurred: {e}")
            return None

# Execute the function
session_key = await fetch_session_key()
print(f"\nFinal session_key: {session_key}")


Step 1: Finding Bahrain 2023 meeting...
Found Bahrain meeting with key: 1141

Step 2: Finding Race session...
Found Race session with key: 7953

Final session_key: 7953
